In [1]:
import pandas as pd
import os
from glob import glob

### Check your data folder to ensure availability of files to process

In [2]:
os.listdir(f"/home/{os.environ.get('USER')}/setu/examples/sample_data")

['sample_english_pdf.parquet', 'sample_english_crawl.json']

### Run the JSON2Parquet Stage to convert your data jsons into parquets

In [4]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    JSON2ParquetStage \
    --json_glob_path "/home/$USER/setu/examples/sample_data/*.json" \
    --language english \
    --j2p_samples_per_partition 1500 \
    --j2p_verbose False \
    --j2p_run_mode data \
    --j2p_parquet_output_path /home/$USER/setu/examples/output/j2p_output

24/03/07 01:10:38 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/07 01:10:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/07 01:10:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Command used to run this script:  /home/shanks/setu/setu/run.py --config /home/shanks/setu/configs/crawls/spark_english_config.json --mode crawl --run_local True JSON2ParquetStage --json_glob_path /home/shanks/setu/examples/sample_data/*.json --language english --j2p_samples_per_partition 1500 --j2p_verbose False --j2p_run_mode data --j2p_parquet_output_path /home/shanks/setu/examples/output/j2p_output
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------

### Check the JSON2Parquet Stage Output

In [5]:
json_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/j2p_output"

In [6]:
parquets = glob(f"{json_output_path}/*.parquet")

In [7]:
df = pd.read_parquet(parquets[0])

In [8]:
df.head()

,doc_id,url,source,language,text,timestamp
0,8bc919b9-c288-42fd-b503-d0f366e06a14,https://www.91mobiles.com/nokia-t10-lte-price-...,91mobiles,english,"<html lang=""en"">\n<head>\n \n <m...",10/07/23 10:54


### Run the Text Extraction State to extract text from the previous stage output

In [9]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    ExtractTextStage \
    --te_parquets_path "/home/$USER/setu/examples/output/j2p_output/*.parquet" \
    --te_samples_per_partition 1500 \
    --te_run_mode data \
    --te_output_path "/home/$USER/setu/examples/output/te_output"


24/03/07 01:13:07 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/07 01:13:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/07 01:13:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Command used to run this script:  /home/shanks/setu/setu/run.py --config /home/shanks/setu/configs/crawls/spark_english_config.json --mode crawl --run_local True ExtractTextStage --te_parquets_path /home/shanks/setu/examples/output/j2p_output/*.parquet --te_samples_per_partition 1500 --te_run_mode data --te_output_path /home/shanks/setu/examples/output/te_output
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
----------------

### Check the TextExtract Stage Output

In [10]:
te_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/te_output"

In [12]:
parquets = glob(f"{te_output_path}/*.parquet")

In [13]:
df = pd.read_parquet(parquets[0])

In [14]:
df.head()

,doc_id,url,source,timestamp,successful_extraction,title,description,text,comments,author,...,categories,tags,fingerprint,id,license,body,commentsbody,raw_text,image,pagetype
0,8bc919b9-c288-42fd-b503-d0f366e06a14,https://www.91mobiles.com/nokia-t10-lte-price-...,91mobiles,10/07/23 10:54,True,Nokia T10 LTE,None,"1. Trusted Brand\n2. Originally from Finland, ...",,None,...,[],[],None,None,None,None,None,None,None,None


### Run DocCleanStage on the Text Extraction Stage output

In [18]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    DocCleanStage \
    --doc_df_parquets_path "/home/$USER/setu/examples/output/te_output/*.parquet" \
    --is_doc_df_path_batched False \
    --doc_clean_additional_cols_to_use "url,source,language" \
    --use_symbol_filter True \
    --doc_clean_samples_per_partition 1500 \
    --doc_clean_verbose False \
    --doc_clean_run_mode data \
    --save_symbol_heavy_docs True \
    --symbol_filter_output_path "/home/$USER/setu/examples/output/symbol_filter/" \
    --cleaned_doc_output_path "/home/$USER/setu/examples/output/cleaned_docs/"

24/03/07 01:55:17 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/07 01:55:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/07 01:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Command used to run this script:  /home/shanks/setu/setu/run.py --config /home/shanks/setu/configs/crawls/spark_english_config.json --mode crawl --run_local True DocCleanStage --doc_df_parquets_path /home/shanks/setu/examples/output/te_output/*.parquet --is_doc_df_path_batched False --doc_clean_additional_cols_to_use url,source,language --use_symbol_filter True --doc_clean_samples_per_partition 1500 --doc_clean_verbose False --doc_clean_run_mode data --save_symbol_heavy_docs True --symbol_filter_output_path /home/shanks/setu/examples/output/symbol_filter/ --cleaned_doc_output_path /home/shanks/setu/examples/output/